# Download automatizado de lotes

## Introdução

Os arquivos de lotes estão divididos por sub-prefeituras. Isso acontece em outros casos e é possível adaptar esse script para solucionar dificuldades similares.
É comum precisarmos desse arquivo para fazer alguma análise e sempre acaba ficando a dificuldade de se obeter o conjunto total de feições.

Portanto um robo automatizado que faça o download dos arquivos de maneira consistente pode ser uma ferramenta útil de trabalho para vc que precisa trabalhar com dados de toda a cidade.

## Metodologia

Para fazermos o download de forma automatizada vamos usar a linguagem de programação Python com o WebDriver Selenium

### Abrindo so site do Geosampa

O primeiro passo é abrir o site do GeoSampa em um WebDriver e simular um usuário fazendo o download de todos os arquivos de lote

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# options = webdriver.ChromeOptions()

# Descomente a linha abaixo caso não precise visualizar o browser
# options.add_argument('headless') 

options.add_argument('window-size=900x1080')

# Inicializando o driver, que é uma instância de um browser
driver = webdriver.Chrome(chrome_options=options)
driver.get("http://geosampa.prefeitura.sp.gov.br/PaginasPublicas/_SBC.aspx")

<ipython-input-3-d02019d27c11>:17: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options)


### Clicando no botão de downloads

In [4]:
#popup = driver.find_element_by_xpath('//*[@id="toPopup"]/div[1]')
#driver.execute_script("arguments[0].click();", popup)

#driver.execute_script('document.getElementById("backgroundPopup").remove()')
driver.implicitly_wait(1)

botao_downloads = driver.find_element_by_xpath('//*[contains(@id, "OpenLayers_Control_Panel")]/div[7]')
botao_downloads.click()

In [5]:
#######

driver.implicitly_wait(1)

from selenium.webdriver.support.select import Select
combo_download = Select(driver.find_element_by_id('cboCamadas'))

In [6]:
list(map(lambda x: x.text, combo_download.options))

['',
 'Cidade Solidária',
 'Limites Administrativos',
 'População',
 'Equipamentos',
 'Transporte',
 'Sistema Viário',
 'Habitação e Edificação',
 'Proteção e Defesa Civil',
 'Infraestrutura Urbana',
 'Verde e Recursos Naturais',
 'Meio Físico',
 'Rural',
 'Cadastro',
 'Legislação Urbana',
 'Patrimônio Cultural',
 'Acessibilidade',
 'Atlas Ambiental',
 'Economia',
 'Imagens Satelites',
 'Mosaico de Ortofotos',
 'Plantas e Mapas Históricos',
 'Articulacao de Imagens',
 'Atendimento ao Cidadão',
 'Proteção aos Aeródromos',
 'Licenciamento Ambiental']

### Escolhendo a camada

Agora precisamos escolher no menu DropDown a opção 'Cadastro' e então a opção 'Lotes'. Quando uma nova janela passa a ser apresentada para podermos escolher 'Shapefiles'

In [7]:
driver.implicitly_wait(1)

from selenium.webdriver.support.select import Select
combo_download = Select(driver.find_element_by_id('cboCamadas'))
combo_download.select_by_visible_text('Cadastro')

driver.implicitly_wait(1)

sub_combo_download = Select(driver.find_element_by_id('cboSubCamadas'))
sub_combo_download.select_by_visible_text('Lotes')

driver.implicitly_wait(1)

link_ok = driver.find_element_by_xpath('//*[@id="1"]/td[2]/a[3]')
link_ok.click()

driver.implicitly_wait(1)

# link_shps = driver.find_element_by_xpath('//*[@id="1"]/td[2]/a[2]')
# link_shps.click()

### Iterando sobre os arquivos disponíveis para download

Nessa tela temos links para os downloads de todos os arquivos. Precisamos portanto iterar sobre eles e guardar esses links em um arquivo para postetiormente fazer o download de cada um deles.

In [8]:
%%time


# abrindo  o arquivo em modo append
f = open("download_lotes_geosampa.txt", "w+")

# iterando sobre os links
while True:
    links_arquivos = driver.find_elements_by_class_name("down")

    for link in links_arquivos:
        print(link)
        if 'SHP_LOTES' in link.get_attribute("href"):
            l = link
#             print(l)
            driver.execute_script("arguments[0].click();", l)
            driver.implicitly_wait(1)
            b = driver.find_element_by_xpath("//*[contains(text(), 'De acordo')]")
            driver.execute_script("arguments[0].click();", b)
            driver.implicitly_wait(1)
#             f.write(link.get_attribute("href")+'\n')
            #link.click()
    if driver.find_element_by_xpath('//*[contains(@id, "next__P_")]').get_attribute('class').find('disabled') < 0:
        n = driver.find_element_by_xpath('//*[contains(@id, "next__P_")]/span')
        driver.execute_script("arguments[0].click();", n)
    else:
        break

# fechando o arquivo        
f.close()

<selenium.webdriver.remote.webelement.WebElement (session="d9927db16dac8e7b7fc7096b67fc97ce", element="0.523343106899089-32")>
<selenium.webdriver.remote.webelement.WebElement (session="d9927db16dac8e7b7fc7096b67fc97ce", element="0.523343106899089-33")>
<selenium.webdriver.remote.webelement.WebElement (session="d9927db16dac8e7b7fc7096b67fc97ce", element="0.523343106899089-34")>
<selenium.webdriver.remote.webelement.WebElement (session="d9927db16dac8e7b7fc7096b67fc97ce", element="0.523343106899089-35")>
<selenium.webdriver.remote.webelement.WebElement (session="d9927db16dac8e7b7fc7096b67fc97ce", element="0.523343106899089-36")>
<selenium.webdriver.remote.webelement.WebElement (session="d9927db16dac8e7b7fc7096b67fc97ce", element="0.523343106899089-37")>
<selenium.webdriver.remote.webelement.WebElement (session="d9927db16dac8e7b7fc7096b67fc97ce", element="0.523343106899089-38")>
<selenium.webdriver.remote.webelement.WebElement (session="d9927db16dac8e7b7fc7096b67fc97ce", element="0.523343

<selenium.webdriver.remote.webelement.WebElement (session="d9927db16dac8e7b7fc7096b67fc97ce", element="0.523343106899089-163")>
<selenium.webdriver.remote.webelement.WebElement (session="d9927db16dac8e7b7fc7096b67fc97ce", element="0.523343106899089-164")>
<selenium.webdriver.remote.webelement.WebElement (session="d9927db16dac8e7b7fc7096b67fc97ce", element="0.523343106899089-165")>
<selenium.webdriver.remote.webelement.WebElement (session="d9927db16dac8e7b7fc7096b67fc97ce", element="0.523343106899089-166")>
<selenium.webdriver.remote.webelement.WebElement (session="d9927db16dac8e7b7fc7096b67fc97ce", element="0.523343106899089-167")>
<selenium.webdriver.remote.webelement.WebElement (session="d9927db16dac8e7b7fc7096b67fc97ce", element="0.523343106899089-168")>
<selenium.webdriver.remote.webelement.WebElement (session="d9927db16dac8e7b7fc7096b67fc97ce", element="0.523343106899089-169")>
<selenium.webdriver.remote.webelement.WebElement (session="d9927db16dac8e7b7fc7096b67fc97ce", element="0